In [1]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
#general purpose packages

# nlp
from bs4 import BeautifulSoup
import nltk,string
import re
from nltk.tokenize import sent_tokenize # tokenizes sentences
from nltk.stem import PorterStemmer     # parsing/stemmer
from nltk.tag import pos_tag            # parts-of-speech tagging
from nltk.corpus import wordnet         # sentiment scores
from nltk.stem import WordNetLemmatizer # stem and context
from nltk.corpus import stopwords       # stopwords
from nltk.util import ngrams            # ngram iterator
from nltk.tokenize import word_tokenize

# import word2vec|
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec

# import sklearn
import lightgbm
from lightgbm import LGBMClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
#load data
train_data = pd.read_csv('C:/Users/whtwht97/Desktop/Document/UChicago/Q2 - Health Analytics/final proj/drugsComTrain_raw.csv')
test_data = pd.read_csv('C:/Users/whtwht97/Desktop/Document/UChicago/Q2 - Health Analytics/final proj/drugsComTest_raw.csv')
df = pd.concat([train_data,test_data])
df

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37
...,...,...,...,...,...,...,...
53761,159999,Tamoxifen,"Breast Cancer, Prevention","""I have taken Tamoxifen for 5 years. Side effe...",10,13-Sep-14,43
53762,140714,Escitalopram,Anxiety,"""I&#039;ve been taking Lexapro (escitaploprgra...",9,8-Oct-16,11
53763,130945,Levonorgestrel,Birth Control,"""I&#039;m married, 34 years old and I have no ...",8,15-Nov-10,7
53764,47656,Tapentadol,Pain,"""I was prescribed Nucynta for severe neck/shou...",1,28-Nov-11,20


In [3]:
df.drop_duplicates(subset=["review"],inplace=True)

In [4]:
#create sentiment labels
df["sentiment"] = np.where(df["rating"] > 7, 1, np.where(df["rating"] < 4, -1, 0))

In [5]:
ps = PorterStemmer()
wnl = WordNetLemmatizer()
eng_stopwords = set(stopwords.words("english"))

def review_cleaner(review, lemmatize=True, stem=False):
    '''
        Clean and preprocess a review.
            1. Remove HTML tags
            2. Extract emoticons
            3. Use regex to remove all special characters (only keep letters)
            4. Make strings to lower case and tokenize / word split reviews
            5. Remove English stopwords
            6. Lemmatize
            7. Rejoin to one string
        
        @review (type:str) is an unprocessed review string
        @return (type:str) is a 6-step preprocessed review string
    '''
    if lemmatize == True and stem == True:
        raise RuntimeError("May not pass both lemmatize and stem flags")

    #1. Remove HTML tags
    review = BeautifulSoup(review, "lxml").text

    #2. Use regex to find emoticons
    pattern2 ='(?::|;|=)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(pattern2,review)

    #3. Remove punctuation
    pattern3 = r"[^\w+ \d+]"
    review = re.sub(pattern3, "",review)

    #4. Tokenize into words (all lower case)
    review = review.lower()

    #5. Remove stopwords, Lemmatize, Stem
    word_tokens = word_tokenize(review)
    filtered_sentence = [w for w in word_tokens if not w in eng_stopwords]
    filtered_sentence = []
    for w in word_tokens:
         if w not in eng_stopwords:               
                w = wnl.lemmatize(w)              
                filtered_sentence.append(w)
                  
    #6. Join the review to one sentence
    filtered_sentence = " ".join(filtered_sentence+emoticons)        
    
    return filtered_sentence

In [6]:
df["review"] = df["review"].apply(review_cleaner)

In [7]:
result = []
for text in df.review:
        temp = len(text.split())
        result.append(temp)
        
df["text_len"] = result
df["text_len"].max()

946

In [8]:
df = df[df["text_len"]<=150]

In [9]:
test_data = df[df.uniqueID.isin(test_data["uniqueID"].values)].reset_index(drop=True)
train_data = df[df.uniqueID.isin(train_data["uniqueID"].values)].reset_index(drop=True)

In [10]:
#assign x and y
train_x = train_data["review"]
test_x = test_data["review"]
train_y = train_data["sentiment"]
test_y = test_data["sentiment"]

In [11]:
# CountVectorizer fit and transform
clf = CountVectorizer(ngram_range=(1,4),max_df=0.9)
X_train_cv =  clf.fit_transform(train_x)
X_test_cv = clf.transform(test_x)

In [12]:
X_train_cv = X_train_cv.astype("float64")
train_y = train_y.astype("float64")
X_test_cv = X_test_cv.astype("float64")

In [17]:
# LIGHTGBM
gbm_clf = lightgbm.LGBMClassifier(learning_rate=0.2,num_leaves=40,max_depth=20)
gbm_clf.fit(X_train_cv, train_y)
gbm_pred = gbm_clf.predict(X_test_cv)

In [18]:
print('\tClassification Report for LightGBM:\n\n',classification_report(test_y,gbm_pred, target_names=['Negative', 'Neutral', 'Positive']))

	Classification Report for LightGBM:

               precision    recall  f1-score   support

    Negative       0.68      0.63      0.65      3539
     Neutral       0.44      0.18      0.25      2819
    Positive       0.77      0.92      0.84      9780

    accuracy                           0.73     16138
   macro avg       0.63      0.58      0.58     16138
weighted avg       0.69      0.73      0.69     16138

